In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json
from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [2]:
def fun(x):
    if x < -50:
        return 'Strong Sell'
    elif x > -50  and x < -10:
        return 'Sell'
    elif x > -10 and x < 10:
        return 'Neutral'
    elif x > 10 and x < 50:
        return 'Buy'
    else:
        return 'Strong Buy'
    
def fun1(x,v1,v2,v3,v4):
    if x < v1:
        return 'Strong Sell'
    elif x > v1 and x < v2:
        return 'Sell'
    elif x > v2 and x < v3:
        return 'Neutral'
    elif x > v3 and x < v4:
        return 'Buy'
    else:
        return 'Strong Buy'
    
    
def acc(x):
    if x.name == 'Strong Sell' or x.name == 'Sell':
        return (x['Strong Sell'] + x['Sell']) / x['Total']
    elif x.name == 'Neutral':
        return (x['Neutral']) / x['Total']
    else:
        return (x['Strong Buy'] + x['Buy']) / x['Total']

In [12]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
candlestick = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    
    query = '''SELECT * FROM ianbalina.ico_technical_analyses;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.technical_analysis_scoring;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.technical_analysis_weights;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    query = '''SELECT * FROM ianbalina.ico_candlestick_patterns;'''
    data = pd.read_sql_query(query, conn)
    candlestick = pd.concat([candlestick,data])
    
    conn.close() 

I0926 09:11:29.594898 15768 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I0926 09:11:32.458302 15768 transport.py:1819] Authentication (password) successful!


In [44]:
df['ico_id'][df['ico_symbol'] == 'QTUM'].unique()

array([3323], dtype=int64)

In [13]:
df['ico_symbol'].nunique()

333

In [14]:
dates = d1['date'].unique()

In [21]:
dates[-14],dates[-7]

(datetime.date(2019, 9, 13), datetime.date(2019, 9, 20))

In [18]:
dates[-8],dates[-1]

(datetime.date(2019, 9, 19), datetime.date(2019, 9, 26))

In [15]:
dates

array([datetime.date(2019, 7, 29), datetime.date(2019, 7, 31),
       datetime.date(2019, 8, 5), datetime.date(2019, 8, 6),
       datetime.date(2019, 8, 14), datetime.date(2019, 8, 19),
       datetime.date(2019, 8, 21), datetime.date(2019, 8, 23),
       datetime.date(2019, 8, 26), datetime.date(2019, 8, 28),
       datetime.date(2019, 8, 30), datetime.date(2019, 9, 1),
       datetime.date(2019, 9, 2), datetime.date(2019, 9, 4),
       datetime.date(2019, 9, 5), datetime.date(2019, 9, 6),
       datetime.date(2019, 9, 7), datetime.date(2019, 9, 8),
       datetime.date(2019, 9, 9), datetime.date(2019, 9, 10),
       datetime.date(2019, 9, 11), datetime.date(2019, 9, 12),
       datetime.date(2019, 9, 13), datetime.date(2019, 9, 14),
       datetime.date(2019, 9, 15), datetime.date(2019, 9, 16),
       datetime.date(2019, 9, 17), datetime.date(2019, 9, 18),
       datetime.date(2019, 9, 19), datetime.date(2019, 9, 20),
       datetime.date(2019, 9, 21), datetime.date(2019, 9, 22),
  

#### Date:  31-07 to 30-08

In [27]:
d3['indicator'].iloc[11] = 'ichimoku_lagging_span'
candlestick['signal'] = candlestick['signal'].replace(['bullish','bearish','bearish_bullish'],[25,-25,0])
d1 = d1[d1['date'] == dates[-14]]

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [23]:
t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= dates[-14]][t1['date'] <= dates[-7]]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        x1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        x2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,x1,x2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

#### Old weights

In [28]:
d3['weight'] = [3, 13, 13, 22, 8, 5, 21, 3, 0, 22, 4, 7, 13, 21]
d4 = pd.merge(d1,d3[['indicator','weight']], on=['indicator'] ,how='left')
d4['signal'] = d4['signal'].replace(['neutral','downward','upward','buy','sell','bullish','bearish'],[0,-25,25,75,-75,25,-25])

In [30]:
d5

,indicator,weight,signal
234,adx,3.000000,0
235,ao,13.000000,0
236,cci,13.000000,0
237,macd,22.000000,0
238,rsi,5.000000,0
239,kd,21.000000,0
240,wr,3.000000,0
241,roc,0.000000,0
242,mfi,21.000000,0
243,trix,22.000000,0


In [29]:
temp = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    d5 = d4[d4['ico_id'].isin(num)]
    d5 = d5[['indicator','weight','signal']]

    candlestick['weight'] = 1
    t1 = candlestick[candlestick['ico_id'].isin(num)]
    t1 = t1[['pattern','weight','signal']]
    t1.columns = d5.columns

    d5 = pd.concat([d5,t1])

    d5 = d5.drop_duplicates('indicator')
    d5['weight'] = d5['weight'].fillna(1)
    d5 = d5.fillna(0)
    temp = pd.concat([temp,pd.DataFrame([coin,sum(d5['weight'] * d5['signal'].astype('float64')) / d5['weight'].sum()])], axis=1)


temp = temp.T
temp.columns = ['Coins','Technical Score']
temp.dropna(inplace=True)


final = pd.merge(temp,t2)

In [31]:
final

,Coins,Technical Score,Price Change,% Change
0,BTC,-7.436709,-49.201172,-0.481486
1,ETH,0.159236,25.818695,13.503195
2,AVT,3.846154,0.020970,15.697842
3,OMG,-16.401274,0.090516,8.625409
4,EOS,22.596154,-0.091381,-2.281257
5,LTC,-6.451613,3.954597,5.610811
6,USDT,13.924051,-0.005540,-0.550518
7,XMR,-17.147436,0.668648,0.916312
8,DASH,-6.570513,7.176437,8.071291
9,NEO,-16.346154,0.471124,5.241699


In [32]:
final['TA result'] = final['Technical Score'].apply(fun)
final['Actual result'] = final['% Change'].apply(fun1,args=[-15,-5,5,15])
final = final[final['Technical Score'] != 0]

In [33]:
data = pd.crosstab(final['TA result'], final['Actual result'])

data = data.reindex(['Strong Sell','Sell','Neutral','Buy','Strong Buy'])
data = data.reindex(columns=['Strong Sell','Sell','Neutral','Buy','Strong Buy'])

x1 = data.sum(axis=1)
data['Total'] = x1
x2 = data.T.apply(acc) * 100
data['Result'] = (x1*x2)/100
data['% Accurate'] = x2
data.fillna(0,inplace=True)

data.index = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%']
data.columns = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%', 'Total', 'Result', '% Accurate']

data

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,1.000000,7.000000,43.000000,20.000000,10.000000,81.000000,8.000000,9.876543
(-5 - 5)%,6.000000,25.000000,109.000000,63.000000,30.000000,233.000000,109.000000,46.781116
(5 - 15)%,0.000000,1.000000,9.000000,5.000000,0.000000,15.000000,5.000000,33.333333
(15)%,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [34]:
data['Result'].sum() / data['Total'].sum()

0.3696969696969697

#### New Weights

In [45]:
df[df['ico_id'] == 3323]

,id,ico_id,ico_symbol,date,open,close,low,high,created_at,updated_at,volume,btc_open,btc_close,btc_low,btc_high,value_from_coinmarketcap
3245,224964,3323,QTUM,2018-02-13,26.195000,28.990000,26.195000,29.930000,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3246,224965,3323,QTUM,2018-02-14,29.299999,28.850000,26.700001,29.327999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3247,224966,3323,QTUM,2018-02-15,28.594999,31.850000,28.594999,32.209999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3248,224967,3323,QTUM,2018-02-16,31.850000,31.789000,30.500000,32.987999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3249,224968,3323,QTUM,2018-02-17,31.778000,31.940001,30.350000,32.678001,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3250,224969,3323,QTUM,2018-02-18,31.764999,32.410999,31.471001,32.882999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3251,224970,3323,QTUM,2018-02-19,32.410000,29.860001,28.511000,32.689999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3252,224971,3323,QTUM,2018-02-20,29.410000,31.400000,29.222000,32.000000,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3253,224972,3323,QTUM,2018-02-21,31.190001,28.740000,28.551001,32.279999,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0
3254,224973,3323,QTUM,2018-02-22,28.740000,26.559000,25.711000,29.249001,2019-06-24 16:47:10,2019-06-24 16:47:10,nan,nan,nan,nan,nan,0


In [38]:
d3['weight'] = [11, 18, 2, 4, 8, 0, 23, 12, 2, 6, 21, 1, 1, 4]
d4 = pd.merge(d1,d3[['indicator','weight']], on=['indicator'] ,how='left')
d4['signal'] = d4['signal'].replace(['neutral','downward','upward','buy','sell','bullish','bearish'],[0,-25,25,75,-75,25,-25])

In [39]:
temp = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    d5 = d4[d4['ico_id'].isin(num)]
    d5 = d5[['indicator','weight','signal']]

    candlestick['weight'] = 1
    t1 = candlestick[candlestick['ico_id'].isin(num)]
    t1 = t1[['pattern','weight','signal']]
    t1.columns = d5.columns

    d5 = pd.concat([d5,t1])

    d5 = d5.drop_duplicates('indicator')
    d5['weight'] = d5['weight'].fillna(1)
    d5 = d5.fillna(0)
    temp = pd.concat([temp,pd.DataFrame([coin,sum(d5['weight'] * d5['signal'].astype('float64')) / d5['weight'].sum()])], axis=1)


temp = temp.T
temp.columns = ['Coins','Technical Score']
temp.dropna(inplace=True)


final = pd.merge(temp,t2)

final['TA result'] = final['Technical Score'].apply(fun)
final['Actual result'] = final['% Change'].apply(fun1,args=[-15,-5,5,15])
final = final[final['Technical Score'] != 0]
data = pd.crosstab(final['TA result'], final['Actual result'])

data = data.reindex(['Strong Sell','Sell','Neutral','Buy','Strong Buy'])
data = data.reindex(columns=['Strong Sell','Sell','Neutral','Buy','Strong Buy'])

x1 = data.sum(axis=1)
data['Total'] = x1
x2 = data.T.apply(acc) * 100
data['Result'] = (x1*x2)/100
data['% Accurate'] = x2
data.fillna(0,inplace=True)

data.index = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%']
data.columns = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%', 'Total', 'Result', '% Accurate']

data

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,10.000000,70.000000,44.000000,13.000000,137.000000,10.000000,7.299270
(-5 - 5)%,7.000000,22.000000,84.000000,40.000000,26.000000,179.000000,84.000000,46.927374
(5 - 15)%,0.000000,3.000000,6.000000,4.000000,1.000000,14.000000,5.000000,35.714286
(15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
data['Result'].sum() / data['Total'].sum()

0.3

#### Weekly

In [32]:
d3['indicator'].iloc[11] = 'ichimoku_lagging_span'
candlestick['signal'] = candlestick['signal'].replace(['bullish','bearish','bearish_bullish'],[25,-25,0])
d1 = d1[d1['date'] == datetime.date(2019,8,19)]

In [38]:
t2 = pd.DataFrame()
for coin in df['ico_symbol'].unique().tolist():
    t1 = df[df['ico_symbol'] == coin]
    t1 = t1[t1['date'] >= datetime.date(2019,8,19)][t1['date'] <= datetime.date(2019,8,27)]
    t1 = t1.drop_duplicates(subset=['date'])
    t1 = t1.sort_values('date')
    if len(t1) > 1:
        x1 = t1['close'].iloc[-1] - t1['close'].iloc[0]
        x2 = (t1['close'].iloc[-1] - t1['close'].iloc[0]) / t1['close'].iloc[0] * 100
        t2 = pd.concat([t2,pd.DataFrame([coin,x1,x2])], axis=1)

t2 = t2.T
t2.columns = ['Coins','Price Change', '% Change']

In [42]:
def get_result(x = [4,2,1,11,1,3,7,2,1,10,7,21,25,6]):
    d3['weight'] = x
    d4 = pd.merge(d1,d3[['indicator','weight']], on=['indicator'] ,how='left')
    d4['signal'] = d4['signal'].replace(['neutral','downward','upward','buy','sell'],[0,-25,25,75,-75])

    
    temp = pd.DataFrame()
    for coin in df['ico_symbol'].unique().tolist():
        num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
        d5 = d4[d4['ico_id'].isin(num)]
        d5 = d5[['indicator','weight','signal']]

        candlestick['weight'] = 1
        t1 = candlestick[candlestick['ico_id'].isin(num)]
        t1 = t1[['pattern','weight','signal']]
        t1.columns = d5.columns

        d5 = pd.concat([d5,t1])

        d5 = d5.drop_duplicates('indicator')
        d5['weight'] = d5['weight'].fillna(1)
        d5 = d5.fillna(0)
        temp = pd.concat([temp,pd.DataFrame([coin,sum(d5['weight'] * d5['signal'].astype('float64')) / d5['weight'].sum()])], axis=1)
        
        
    temp = temp.T
    temp.columns = ['Coins','Technical Score']
    temp.dropna(inplace=True)

    
    final = pd.merge(temp,t2)
    
    final['TA result'] = final['Technical Score'].apply(fun)
    final['Actual result'] = final['% Change'].apply(fun1,args=[-15,-5,5,15])
    final = final[final['Technical Score'] != 0]
    data = pd.crosstab(final['TA result'], final['Actual result'])

    data = data.reindex(['Strong Sell','Sell','Neutral','Buy','Strong Buy'])
    data = data.reindex(columns=['Strong Sell','Sell','Neutral','Buy','Strong Buy'])

    x1 = data.sum(axis=1)
    data['Total'] = x1
    x2 = data.T.apply(acc) * 100
    data['Result'] = (x1*x2)/100
    data['% Accurate'] = x2
    data.fillna(0,inplace=True)
    
    data.index = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%']
    data.columns = ['(-15)%','(-15 - -5)%', '(-5 - 5)%', '(5 - 15)%', '(15)%', 'Total', 'Result', '% Accurate']

    return data

In [43]:
d3['weight'] = [3, 13, 13, 22, 8, 5, 21, 3, 0, 22, 4, 7, 13, 21]
d3

,id,indicator,weight,created_at,updated_at
0,1,adx,3,None,None
1,2,ao,13,None,None
2,3,cci,13,None,None
3,4,macd,22,None,None
4,5,fi,8,None,None
5,6,rsi,5,None,None
6,7,kd,21,None,None
7,8,wr,3,None,None
8,9,roc,0,None,None
9,10,trix,22,None,None


In [44]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,3.000000,2.000000,5.000000,0.000000,1.000000,11.000000,5.000000,45.454545
(-5 - 5)%,10.000000,15.000000,77.000000,11.000000,4.000000,117.000000,77.000000,65.811966
(5 - 15)%,6.000000,35.000000,62.000000,12.000000,12.000000,127.000000,24.000000,18.897638
(15)%,1.000000,1.000000,2.000000,0.000000,1.000000,5.000000,1.000000,20.000000


In [45]:
x['Result'].sum() / x['Total'].sum()

0.41153846153846146

In [46]:
d3['weight'] = [11, 3, 4, 15, 5, 3, 5, 1, 23, 15, 14, 19, 2, 19]
d3

,id,indicator,weight,created_at,updated_at
0,1,adx,11,None,None
1,2,ao,3,None,None
2,3,cci,4,None,None
3,4,macd,15,None,None
4,5,fi,5,None,None
5,6,rsi,3,None,None
6,7,kd,5,None,None
7,8,wr,1,None,None
8,9,roc,23,None,None
9,10,trix,15,None,None


In [47]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
(-5 - 5)%,16.000000,49.000000,138.000000,21.000000,17.000000,241.000000,138.000000,57.261411
(5 - 15)%,3.000000,3.000000,6.000000,2.000000,0.000000,14.000000,2.000000,14.285714
(15)%,1.000000,1.000000,1.000000,0.000000,1.000000,4.000000,1.000000,25.000000


In [48]:
x['Result'].sum() / x['Total'].sum()

0.5423076923076922

In [49]:
d3['weight'] = [11, 3, 4, 15, 5, 3, 5, 1, 23, 15, 14, 19, 2, 19]
d3

,id,indicator,weight,created_at,updated_at
0,1,adx,11,None,None
1,2,ao,3,None,None
2,3,cci,4,None,None
3,4,macd,15,None,None
4,5,fi,5,None,None
5,6,rsi,3,None,None
6,7,kd,5,None,None
7,8,wr,1,None,None
8,9,roc,23,None,None
9,10,trix,15,None,None


In [50]:
x = get_result(d3['weight'])
x

,(-15)%,(-15 - -5)%,(-5 - 5)%,(5 - 15)%,(15)%,Total,Result,% Accurate
(-15)%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
(-15 - -5)%,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
(-5 - 5)%,16.000000,49.000000,138.000000,21.000000,17.000000,241.000000,138.000000,57.261411
(5 - 15)%,3.000000,3.000000,6.000000,2.000000,0.000000,14.000000,2.000000,14.285714
(15)%,1.000000,1.000000,1.000000,0.000000,1.000000,4.000000,1.000000,25.000000


In [51]:
x['Result'].sum() / x['Total'].sum()

0.5423076923076922

#### temp

In [64]:
t = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.predictions_meta;'''
    data = pd.read_sql_query(query, conn)
    t = pd.concat([t,data])
    
    conn.close() 

I0923 08:49:39.487410  6556 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I0923 08:49:41.258848  6556 transport.py:1819] Authentication (password) successful!


In [66]:
df

,id,ico_id,ico_symbol,date,open,close,low,high,created_at,updated_at,volume,btc_open,btc_close,btc_low,btc_high,value_from_coinmarketcap
0,994,1,BTC,2018-01-08,17169.000000,16215.357422,15712.000000,17178.000000,2019-05-19 19:42:33,2019-05-19 19:42:33,nan,nan,nan,nan,nan,0
1,995,1,BTC,2018-01-09,16213.000000,14906.000000,13755.000000,16329.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
2,996,1,BTC,2018-01-10,14911.000000,14430.000000,14122.000000,15355.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
3,997,1,BTC,2018-01-11,14435.000000,14900.000000,13300.000000,14901.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
4,998,1,BTC,2018-01-12,14900.000000,13254.000000,12639.000000,14999.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
5,999,1,BTC,2018-01-13,13253.000000,13790.000000,12751.000000,14100.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
6,1000,1,BTC,2018-01-14,13790.000000,14199.000000,13750.000000,14580.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
7,1001,1,BTC,2018-01-15,14192.000000,13558.000000,12874.341797,14393.000000,2019-05-19 19:42:34,2019-05-19 19:42:34,nan,nan,nan,nan,nan,0
8,1492,2,ETH,2018-01-11,1290.010010,1247.000000,1207.609985,1386.989990,2019-05-19 19:42:39,2019-05-19 19:42:39,nan,nan,nan,nan,nan,0
9,1493,2,ETH,2018-01-12,1247.000000,1137.010010,1083.969971,1335.000000,2019-05-19 19:42:39,2019-05-19 19:42:39,nan,nan,nan,nan,nan,0


In [69]:
t1 = []
for ico in t['ico_id']:
    t1.append(df['ico_symbol'][df['ico_id'] == ico].iloc[0])

In [121]:
t

,id,ico_id,content,latest,created_at,updated_at
0,1,2987,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:43,2019-09-04 15:36:43
1,2,3146,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:49,2019-09-04 15:36:49
2,3,3181,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:52,2019-09-04 15:36:52
3,4,3217,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:55,2019-09-04 15:36:55
4,5,3312,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:58,2019-09-04 15:36:58
5,6,3317,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:36:59,2019-09-04 15:36:59
6,7,3335,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:37:01,2019-09-04 15:37:01
7,8,3369,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:37:03,2019-09-04 15:37:03
8,9,3376,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:37:04,2019-09-04 15:37:04
9,10,3379,"{""time"":[""last month-2019"",""2nd last month-201...",0,2019-09-04 15:37:04,2019-09-04 15:37:04


In [105]:
t2 = pd.DataFrame()
for i in range(len(t)):
    x = t['content'].iloc[i]
    x = eval(x)['accuracy']
    t2 = pd.concat([t2, pd.DataFrame(x).T])

In [107]:
t2['Coin'] = t1

In [112]:
t2.columns = ["last month-2019","2nd last month-2019","3rd last month-2019","Coin"]

In [116]:
sum(t2['last month-2019'] > 0.90)

334

In [118]:
len(t2)

334

In [120]:
len(t2.drop_duplicates())

61

In [110]:
x = t['content'].iloc[0]

In [111]:
xfg cbtd5r

'{"time":["last month-2019","2nd last month-2019","3rd last month-2019"],"rmse":[0.16757141547289078,3.891598099814588,1.7960051879596932],"mae":[0.13416646342124405,3.5354556051898527,1.3837626131216327],"accuracy":[0.955728906445522,0.5409574821382895,0.7165221023930449]}'

In [129]:
t = pd.read_csv('temp.csv')

In [142]:
t1 = pd.DataFrame()
for val in range(0,len(t),4):
    t2 = t[val:val+4]
    t2 = t2.iloc[:,1].tolist()
    t1 = pd.concat([t1,pd.DataFrame(t2).T])

In [144]:
len(t1)

114

In [146]:
t1.columns = ["last month-2019","2nd last month-2019","3rd last month-2019","Coin"]

In [152]:
sum(t1['last month-2019'].astype('float64') > 0.9)

37

In [153]:
sum(t1['last month-2019'].astype('float64') > 0.8)

78

In [157]:
t1.to_csv('data.csv', index=False)